This Spark notebook will answer the questions of the Amadeus Challenge (bookings + searches) using Spark instead of python pandas.

In [44]:
%AddDeps org.typelevel cats-core_2.11 0.9.0
%AddDeps org.typelevel cats-kernel_2.11 0.9.0
%AddDeps org.typelevel cats-macros_2.11 0.9.0
%AddDeps io.circe circe-core_2.11 0.7.0
%AddDeps io.circe circe-generic_2.11 0.7.0
%AddDeps io.circe circe-jawn_2.11 0.7.0
%AddDeps io.circe circe-numbers_2.11 0.7.0
%AddDeps io.circe circe-parser_2.11 0.7.0
%AddDeps org.spire-math jawn-parser_2.11 0.10.4
%AddDeps org.typelevel machinist_2.11 0.6.1
%AddDeps org.typelevel macro-compat_2.11 1.1.1
%AddDeps com.github.julien-truffaut monocle-core_2.11 1.1.0
%AddDeps com.github.julien-truffaut monocle-macro_2.11 1.1.0
%AddDeps org.scalafx scalafx_2.11-8 0.92-R10
%AddDeps org.scalaz scalaz-core_2.11 7.1.1
%AddDeps com.chuusai shapeless_2.11 2.3.2
%AddDeps com.github.mpilquist simulacrum_2.11 0.10.0
%AddDeps org.webjars.bower vega-3 0.0-rc4
%AddDeps org.webjars.bower vega-lite 1.2.0
%AddDeps org.vegas-viz vegas_2.11 0.3.11
%AddDeps org.vegas-viz vegas-macros_2.11 0.3.11

Marking org.typelevel:cats-core_2.11:0.9.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir9064204515783808255/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir9064204515783808255/toree_add_deps/https/repo1.maven.org/maven2/org/typelevel/cats-core_2.11/0.9.0/cats-core_2.11-0.9.0.jar
Marking org.typelevel:cats-kernel_2.11:0.9.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir9064204515783808255/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir9064204515783808255/toree_add_deps/https/repo1.maven.org/maven2/org/typelevel/cats-kernel_2.11/0.9.0/cats-kernel_2.11-0.9.0.jar
Marking org.typelevel:cats-macros_2.11:0.9.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir9064204515783808255/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir9064204515783808255/toree_add_deps/https/repo1.maven.org/maven2/org/typelevel/cats-macros_2.11/0.9.0/cats-macros_2

In [46]:
%AddDeps org.plotly-scala plotly-jupyter-scala_2.12 0.3.3

Marking org.plotly-scala:plotly-jupyter-scala_2.12:0.3.3 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir9064204515783808255/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir9064204515783808255/toree_add_deps/https/repo1.maven.org/maven2/org/plotly-scala/plotly-jupyter-scala_2.12/0.3.3/plotly-jupyter-scala_2.12-0.3.3.jar


# RDD's

In [7]:
import org.apache.spark.storage.StorageLevel._

# 1) First exercise: count the number of lines in Python for each file

Create manually the SparkContext (this is not needed, by default sc is loaded). Indeed if I try this I have problems...

## Read files
Define paths and load RDDs on searches and bookings, save first line as header then remove first lines from RDD.

In [8]:
val root_path: String = "/media/sf_VirtualFolder/amadeus/";//where to find files
val searches_path: String = root_path + "searches.csv.bz2";
val bookings_path: String = root_path + "bookings.csv.bz2";

root_path = /media/sf_VirtualFolder/amadeus/
searches_path = /media/sf_VirtualFolder/amadeus/searches.csv.bz2
bookings_path = /media/sf_VirtualFolder/amadeus/bookings.csv.bz2


/media/sf_VirtualFolder/amadeus/bookings.csv.bz2

In [9]:
val searches = sc.textFile(searches_path)//Load RDD 1
val bookings = sc.textFile(bookings_path)//Load RDD 2

val headerColumns_searches = searches.first().split("\\^").to[List]
val head_searches = sc.textFile(searches_path).first()

val headerColumns_bookings = bookings.first().split("\\^").to[List]
val head_bookings = sc.textFile(bookings_path).first()

searches = /media/sf_VirtualFolder/amadeus/searches.csv.bz2 MapPartitionsRDD[1] at textFile at <console>:34
bookings = /media/sf_VirtualFolder/amadeus/bookings.csv.bz2 MapPartitionsRDD[3] at textFile at <console>:35
headerColumns_searches = List(Date, Time, TxnCode, OfficeID, Country, Origin, Destination, RoundTrip, NbSegments, Seg1Departure, Seg1Arrival, Seg1Date, Seg1Carrier, Seg1BookingCode, Seg2Departure, Seg2Arrival, Seg2Date, Seg2Carrier, Seg2BookingCode, Seg3Departure, Seg3Arrival, Seg3Date, Seg3Carrier, Seg3BookingCode, Seg4Departure, Seg4Arrival, Seg4Date, Seg4Carrier, Seg4BookingCode, Seg5Departure, Seg5Arrival, Seg5Date, Seg5Carrier, Seg5BookingCode, Seg6Departure, Seg6Arrival, Seg6Date, Seg6Car...


List(Date, Time, TxnCode, OfficeID, Country, Origin, Destination, RoundTrip, NbSegments, Seg1Departure, Seg1Arrival, Seg1Date, Seg1Carrier, Seg1BookingCode, Seg2Departure, Seg2Arrival, Seg2Date, Seg2Carrier, Seg2BookingCode, Seg3Departure, Seg3Arrival, Seg3Date, Seg3Carrier, Seg3BookingCode, Seg4Departure, Seg4Arrival, Seg4Date, Seg4Carrier, Seg4BookingCode, Seg5Departure, Seg5Arrival, Seg5Date, Seg5Carrier, Seg5BookingCode, Seg6Departure, Seg6Arrival, Seg6Date, Seg6Carrier, Seg6BookingCode, From, IsPublishedForNeg, IsFromInternet, IsFromVista, TerminalID, InternetOffice)

In [4]:
head_bookings

act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route          ^carrier^bkg_class^cab_class^brd_time           ^off_time           ^pax^year^month^oid      

In [5]:
val searches_count = searches.count()

[Stage 4:=====================================================>   (14 + 1) / 15]                                            (1 + 4) / 15]

searches_count = 20390199


20390199

In [6]:
val bookings_count = bookings.count()

[Stage 5:=====================================================>   (16 + 1) / 17]

bookings_count = 10000011


10000011

## 2) Top 10 arrival airports in the world in 2013 (using the bookings file)
Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

In [10]:
/* Lets define a function to take each line and return what we want
 * In particular, we want the column 12 with arr_port and 34 with pax
 * VIP: to handle the wrong lines reading (ex. 14 fields) we need to cover that exception, 
 * otherwise it would corrupt the RDD and give many headaches...
 */
def parseLineBookings(line:String) = {
    val fields   = line.split("\\^")
    //check if the line is complete..
    if (fields.length>34){
    val arr_port = fields(12).trim
    val paxstring = fields(34).trim
    if (arr_port=="" || paxstring=="") ("KKK","0") else
    (arr_port, paxstring)
    }
    else ("KKK","0")
}

parseLineBookings: (line: String)(String, String)


In [11]:
//Load the pair RDD from bookings with arr_port and pax
val pair_rdd = bookings.map(parseLineBookings).persist(MEMORY_AND_DISK)

pair_rdd = MapPartitionsRDD[8] at map at <console>:38


MapPartitionsRDD[8] at map at <console>:38

In [12]:
pair_rdd.take(10)//test

[(arr_port,pax), (LHR,-1), (CLT,1), (CLT,1), (SVO,1), (SVO,1), (LGA,1), (LGA,1), (SIN,2), (SIN,2)]

In [13]:
val kk = pair_rdd.filter(_._1 != "arr_port" ).persist(MEMORY_AND_DISK).filter(_._1 == "KKK").collect()//testing filters

[Stage 5:=====================================================>   (16 + 1) / 17]

kk = Array((KKK,0))


[(KKK,0)]

In [14]:
val pair_rdd_good = pair_rdd.filter(_._1 != "arr_port" ).filter(_._1 != "KKK").persist(MEMORY_AND_DISK)//now create filtered pairRDD

pair_rdd_good = MapPartitionsRDD[12] at filter at <console>:39


MapPartitionsRDD[12] at filter at <console>:39

In [15]:
//Define a conversion to integer with exception handling. 
//This is important to avoid NaN when trying to convert your strings to integers
def toInt(s: String): Option[Int] = {
  try {
    Some(s.toInt)
  } catch {
    case e: Exception => None
  }
}

toInt: (s: String)Option[Int]


In [16]:
//We map with some care using match to handle the exceptions. If we find a None we will write 0 pax.
//Make sure wee have integers
val pair_bookings = pair_rdd_good.map(x=> {
      val tst = toInt(x._2)
      tst  match {
      case None => (x._1, 0)
      case Some(y) => (x._1, y) }
}).persist(MEMORY_AND_DISK)

pair_bookings = MapPartitionsRDD[13] at map at <console>:45


MapPartitionsRDD[13] at map at <console>:45

In [17]:
pair_bookings.take(10)//test

[(LHR,-1), (CLT,1), (CLT,1), (SVO,1), (SVO,1), (LGA,1), (LGA,1), (SIN,2), (SIN,2), (SIN,2)]

In [18]:
//reduceByKey will groupByKey and then do the required operation on the values!
val temp = pair_bookings.reduceByKey(_+_)

temp = ShuffledRDD[14] at reduceByKey at <console>:46


ShuffledRDD[14] at reduceByKey at <console>:46

In [19]:
temp.take(5)//test

[Stage 7:=====================================================>   (16 + 1) / 17]

[(IXS,390), (ELS,1370), (YQG,160), (SUV,220), (LFW,370)]

In [20]:
//In order to use the efficient transformation sortByKey we need to swap Keys and Values first.
val top10 = temp.map(x => (x._2,x._1)).sortByKey(false).take(10)

top10 = Array((88809,LHR), (70930,MCO), (70530,LAX), (69630,LAS), (66270,JFK), (64490,CDG), (59460,BKK), (58150,MIA), (58000,SFO), (55590,DXB))


[(88809,LHR), (70930,MCO), (70530,LAX), (69630,LAS), (66270,JFK), (64490,CDG), (59460,BKK), (58150,MIA), (58000,SFO), (55590,DXB)]

In [21]:
top10.foreach(e => println(e))

(88809,LHR)
(70930,MCO)
(70530,LAX)
(69630,LAS)
(66270,JFK)
(64490,CDG)
(59460,BKK)
(58150,MIA)
(58000,SFO)
(55590,DXB)


**We have checked that these results are the same obtained using pandas!**

Now we will load a text file with the information of Airports and cities.

In [22]:
val airDB = sc.textFile("amadeus/GlobalAirportDatabase.txt")

airDB = amadeus/GlobalAirportDatabase.txt MapPartitionsRDD[20] at textFile at <console>:30


amadeus/GlobalAirportDatabase.txt MapPartitionsRDD[20] at textFile at <console>:30

In this case we want just 1 and 3 positions (IATA,CITY)

In [23]:
/*Lets define a function to take each line and return what we want
 * In particular, we want the column 12 with arr_port and 34 with pax
 * VIP: to handle the wrong lines reading (ex. 14 fields) we need to cover that exception, 
 * otherwise it would corrupt the RDD and give many headaches...
 */
def parseLineAirDB(line:String) = {
    val fields   = line.split(':')
    //check if the line is complete..
    if (fields.length>4){
    val IATA = fields(1).trim
    val CITY = fields(3).trim
    (IATA,CITY)
    }
    else ("KKK","NONE")
}

parseLineAirDB: (line: String)(String, String)


In [24]:
val tt = airDB.map(parseLineAirDB).filter(_._1 != "N/A").collect

tt = Array((GKA,GOROKA), (LAE,LAE), (MAG,MADANG), (HGU,MOUNT HAGEN), (LAE,NADZAB), (POM,PORT MORESBY), (RAB,RABAUL), (WWK,WEWAK), (UAK,NARSSARSSUAQ), (JCH,CHRISTIANSHAAB), (JEG,EGEDESMINDE), (JFR,FREDERIKSHAAB), (JGR,GRONNEDAL), (GOH,GODTHAAB), (JGO,GODHAVN), (JHS,HOLSTEINSBORG), (JJU,JULIANEHAAB), (JAV,JAKOBSHAVN), (KUS,KULUSUK), (JNN,NANORTALIK), (JNS,NARSSAQ), (SFJ,SONDRESTROM), (JSU,SUKKERTOPPEN), (THU,THULE), (JUV,UPERNAVIK), (AEY,AKUREYRI), (EGS,EGILSSTADIR), (GRY,GRIMSEY), (HFN,HOFN), (HZK,HUSAVIK), (IFJ,ISAFJORDUR), (KEF,KEFLAVIK), (OPA,KOPASKER), (SAK,SAUDARKROKUR), (NOR,NORDFJORDUR), (PFJ,PATREKSFJORDUR), (RFN,RAUFARHOFN), (RKV,REYKJAVIK), (SIJ,SIGLUFJORDUR), (TEY,THINGEYRI), (THO,THORSHOFN), (VEY,VESTMANNAEYJAR), (VPN,VOPNAFJORDUR), (YQC,QUAQTAQ), (YA...


[(GKA,GOROKA), (LAE,LAE), (MAG,MADANG), (HGU,MOUNT HAGEN), (LAE,NADZAB), (POM,PORT MORESBY), (RAB,RABAUL), (WWK,WEWAK), (UAK,NARSSARSSUAQ), (JCH,CHRISTIANSHAAB), (JEG,EGEDESMINDE), (JFR,FREDERIKSHAAB), (JGR,GRONNEDAL), (GOH,GODTHAAB), (JGO,GODHAVN), (JHS,HOLSTEINSBORG), (JJU,JULIANEHAAB), (JAV,JAKOBSHAVN), (KUS,KULUSUK), (JNN,NANORTALIK), (JNS,NARSSAQ), (SFJ,SONDRESTROM), (JSU,SUKKERTOPPEN), (THU,THULE), (JUV,UPERNAVIK), (AEY,AKUREYRI), (EGS,EGILSSTADIR), (GRY,GRIMSEY), (HFN,HOFN), (HZK,HUSAVIK), (IFJ,ISAFJORDUR), (KEF,KEFLAVIK), (OPA,KOPASKER), (SAK,SAUDARKROKUR), (NOR,NORDFJORDUR), (PFJ,PATREKSFJORDUR), (RFN,RAUFARHOFN), (RKV,REYKJAVIK), (SIJ,SIGLUFJORDUR), (TEY,THINGEYRI), (THO,THORSHOFN), (VEY,VESTMANNAEYJAR), (VPN,VOPNAFJORDUR), (YQC,QUAQTAQ), (YAB,ARCTIC BAY), (YAL,ALERT BAY), (YAM,SAULT SAINTE MARIE), (YAV,WINNIPEG), (YAW,HALIFAX), (YAY,ST. ANTHONY), (YAZ,TOFINO), (YBB,PELLY BAY), (YBC,BAIE COMEAU), (YBG,BAGOTVILLE), (YBK,BAKER LAKE), (YBL,CAMPBELL RIVER), (YBR,BRANDON), (YBT,BR

In [25]:
//Scala calls dicionary MAP
val airDB_pair:Map[String, String] = tt.toMap

airDB_pair = Map(VAA -> VAASA, ROW -> ROSWELL, FTA -> FUTUNA ISLAND, CEB -> CEBU, DLE -> DOLE, ABX -> ALBURY, KSH -> BAKHTARAN, LYX -> LYDD, LGW -> LONDON, BWN -> BRUNEI, GMP -> SEOUL, SCK -> STOCKTON, MJZ -> MIRNYJ, WCH -> CHAITEN, HWO -> HOLLYWOOD, OMR -> ORADEA, MXF -> MONTGOMERY, KAD -> KADUNA, MQF -> MAGNETIOGORSK, SJI -> SAN JOSE, YOW -> OTTAWA, LHW -> WRIGHT, WLH -> WALAHA, SHO -> SOKCH'O, FRU -> BISHKEK, YHU -> MONTREAL, JNN -> NANORTALIK, CPT -> CAPE TOWN, INH -> INHAMBANE, SYR -> SYRACUSE, JMM -> MALMOE, PAZ -> POZA RICO, UTO -> INDIAN MOUNTAINS, ION -> IMPFONDO, XBK -> BOURG, LCC -> LECCE, GDQ -> GONDAR, PDV -> PLOVDIV, MZO -> MANZANILLO, KGA -> KANANGA, IVL -> IVALO, BLA -> BARCELONA, ACY -> ATLANTIC CITY, GMB -> GAMBELLA, PIO -> PISCO, MUC -> MUNICH, SIR...


Map(VAA -> VAASA, ROW -> ROSWELL, FTA -> FUTUNA ISLAND, CEB -> CEBU, DLE -> DOLE, ABX -> ALBURY, KSH -> BAKHTARAN, LYX -> LYDD, LGW -> LONDON, BWN -> BRUNEI, GMP -> SEOUL, SCK -> STOCKTON, MJZ -> MIRNYJ, WCH -> CHAITEN, HWO -> HOLLYWOOD, OMR -> ORADEA, MXF -> MONTGOMERY, KAD -> KADUNA, MQF -> MAGNETIOGORSK, SJI -> SAN JOSE, YOW -> OTTAWA, LHW -> WRIGHT, WLH -> WALAHA, SHO -> SOKCH'O, FRU -> BISHKEK, YHU -> MONTREAL, JNN -> NANORTALIK, CPT -> CAPE TOWN, INH -> INHAMBANE, SYR -> SYRACUSE, JMM -> MALMOE, PAZ -> POZA RICO, UTO -> INDIAN MOUNTAINS, ION -> IMPFONDO, XBK -> BOURG, LCC -> LECCE, GDQ -> GONDAR, PDV -> PLOVDIV, MZO -> MANZANILLO, KGA -> KANANGA, IVL -> IVALO, BLA -> BARCELONA, ACY -> ATLANTIC CITY, GMB -> GAMBELLA, PIO -> PISCO, MUC -> MUNICH, SIR -> SION, EUA -> EUA, SUU -> FAIRFIELD, BJR -> BAHAR DAR, LXA -> LHASA, IRO -> BIRAO, TYR -> TYLER, TNR -> ANTANANARIVO, UIB -> QUIBDO, IPN -> IPATINGA, CUC -> CUCUTA, CHQ -> CHANIA, YGT -> IGLOOLIK, TRR -> TRINCIOMALEE, MMF -> MAMFE, F

In [26]:
airDB_pair("LHR")//test it

LONDON

In [27]:
top10.foreach(e => println(e._1,airDB_pair(e._2)))

(88809,LONDON)
(70930,ORLANDO)
(70530,LOS ANGELES)
(69630,LAS VEGAS)
(66270,NEW YORK)
(64490,PARIS)
(59460,BANGKOK)
(58150,MIAMI)
(58000,SAN FRANCISCO)
(55590,DUBAI)


In [28]:
val top10_human = top10.map(x=> (x._1, airDB_pair(x._2)) )

top10_human = Array((88809,LONDON), (70930,ORLANDO), (70530,LOS ANGELES), (69630,LAS VEGAS), (66270,NEW YORK), (64490,PARIS), (59460,BANGKOK), (58150,MIAMI), (58000,SAN FRANCISCO), (55590,DUBAI))


[(88809,LONDON), (70930,ORLANDO), (70530,LOS ANGELES), (69630,LAS VEGAS), (66270,NEW YORK), (64490,PARIS), (59460,BANGKOK), (58150,MIAMI), (58000,SAN FRANCISCO), (55590,DUBAI)]

In [29]:
top10_human.foreach(println(_))

(88809,LONDON)
(70930,ORLANDO)
(70530,LOS ANGELES)
(69630,LAS VEGAS)
(66270,NEW YORK)
(64490,PARIS)
(59460,BANGKOK)
(58150,MIAMI)
(58000,SAN FRANCISCO)
(55590,DUBAI)


## 3) Plot the monthly number of searches for 3 airports

plot the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona

For the arriving airport, you can use the Destination column in the searches file. Plot a curve for Málaga, another one for Madrid, and another one for Barcelona, in the same figure.

In [30]:
headerColumns_searches

List(Date, Time, TxnCode, OfficeID, Country, Origin, Destination, RoundTrip, NbSegments, Seg1Departure, Seg1Arrival, Seg1Date, Seg1Carrier, Seg1BookingCode, Seg2Departure, Seg2Arrival, Seg2Date, Seg2Carrier, Seg2BookingCode, Seg3Departure, Seg3Arrival, Seg3Date, Seg3Carrier, Seg3BookingCode, Seg4Departure, Seg4Arrival, Seg4Date, Seg4Carrier, Seg4BookingCode, Seg5Departure, Seg5Arrival, Seg5Date, Seg5Carrier, Seg5BookingCode, Seg6Departure, Seg6Arrival, Seg6Date, Seg6Carrier, Seg6BookingCode, From, IsPublishedForNeg, IsFromInternet, IsFromVista, TerminalID, InternetOffice)

In [31]:
/* Lets define a function to take each line and return what we want
 * In particular, we want the column 12 with arr_port and 34 with pax
 * VIP: to handle the wrong lines reading (ex. 14 fields) we need to cover that exception, 
 * otherwise it would corrupt the RDD and give many headaches...
 */
def parseLineSearches(line:String) = {
    val fields   = line.split("\\^")
    //check if the line is complete..
    if (fields.length>34){
    val date = fields(0).trim
    val destination = fields(6).trim
    
    if (destination ==" " || date=="") ("KKK","0") else
    (destination, date)
    }
    else ("KKK","0")
}

parseLineSearches: (line: String)(String, String)


In [32]:
val searches01 = searches.map(parseLineSearches).persist(MEMORY_AND_DISK)

searches01 = MapPartitionsRDD[23] at map at <console>:37


MapPartitionsRDD[23] at map at <console>:37

In [33]:
searches01.take(10)

[(Destination,Date), (AUH,2013-01-01), (MIL,2013-01-01), (SFO,2013-01-01), (ARN,2013-01-01), (MAD,2013-01-01), (BLR,2013-01-01), (PAR,2013-01-01), (DUB,2013-01-01), (ACE,2013-01-01)]

In [34]:
val searches02=searches01.filter(e => e._1 == "MAD" || e._1 == "BCN" || e._1 == "AGP").persist(MEMORY_AND_DISK)

searches02 = MapPartitionsRDD[24] at filter at <console>:39


MapPartitionsRDD[24] at filter at <console>:39

In [35]:
searches02.take(10)

[(MAD,2013-01-01), (AGP,2013-01-01), (BCN,2013-01-01), (AGP,2013-01-01), (MAD,2013-01-01), (BCN,2013-01-01), (BCN,2013-01-01), (BCN,2013-01-01), (BCN,2013-01-01), (BCN,2013-01-01)]

## Handling dates in Scala/Java

In [36]:
val dateFormat = "yyyy-MM-dd"
val dtf = java.time.format.DateTimeFormatter.ofPattern(dateFormat)
//TEST
val dateString = "2013-01-04"
val d = java.time.LocalDate.parse(dateString, dtf)
val year = d.getYear
val ymd = dateString.split("-").map(toInt(_))

dateFormat = yyyy-MM-dd
dtf = Value(YearOfEra,4,19,EXCEEDS_PAD)'-'Value(MonthOfYear,2)'-'Value(DayOfMonth,2)
dateString = 2013-01-04
d = 2013-01-04
year = 2013
ymd = Array(Some(2013), Some(1), Some(4))


[Some(2013), Some(1), Some(4)]

In [37]:
val MAD01 = searches01.filter(e=> e._1 =="MAD").map(x=>x._2.split("-")(1).toInt)//second is month
val BCN01 = searches01.filter(e=> e._1 =="BCN").map(x=>x._2.split("-")(1).toInt)
val AGP01 = searches01.filter(e=> e._1 =="AGP").map(x=>x._2.split("-")(1).toInt)

MAD01 = MapPartitionsRDD[26] at map at <console>:39
BCN01 = MapPartitionsRDD[28] at map at <console>:40
AGP01 = MapPartitionsRDD[30] at map at <console>:41


MapPartitionsRDD[30] at map at <console>:41

In [38]:
MAD01.take(12)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [39]:
//We map with some care using match to handle the exceptions. If we find a None we will write 0 pax.
//Make sure wee have integers
val monthsMAD = MAD01.map(x=>(x,1)).reduceByKey(_+_).persist(MEMORY_AND_DISK)
val monthsBCN = BCN01.map(x=>(x,1)).reduceByKey(_+_).persist(MEMORY_AND_DISK)
val monthsAGP = AGP01.map(x=>(x,1)).reduceByKey(_+_).persist(MEMORY_AND_DISK)

monthsMAD = ShuffledRDD[32] at reduceByKey at <console>:45
monthsBCN = ShuffledRDD[34] at reduceByKey at <console>:46
monthsAGP = ShuffledRDD[36] at reduceByKey at <console>:47


ShuffledRDD[36] at reduceByKey at <console>:47

In [40]:
val MAD = monthsMAD.collect
val BCN = monthsBCN.collect
val AGP = monthsAGP.collect

[Stage 22:====================================================>   (14 + 1) / 15]

MAD = Array((1,24258), (2,22800), (3,24681), (4,25251), (5,26334), (6,22800), (7,22971), (8,21831), (9,21147), (10,22294), (11,20272), (12,14504))
BCN = Array((1,29469), (2,28329), (3,30552), (4,31236), (5,28728), (6,26505), (7,29241), (8,27075), (9,23427), (10,20276), (11,19824), (12,15400))
AGP = Array((1,9633), (2,8379), (3,10659), (4,8265), (5,10830), (6,7923), (7,8892), (8,7866), (9,8151), (10,6499), (11,6384), (12,3696))


[(1,9633), (2,8379), (3,10659), (4,8265), (5,10830), (6,7923), (7,8892), (8,7866), (9,8151), (10,6499), (11,6384), (12,3696)]

In [60]:
import vegas._
implicit val render = vegas.render.ShowHTML(kernel.display.content("text/html", _))

render = <function1>


<function1>

In [112]:
def loadHistogram(path: String) : List[Tuple2[String, String]] = {
  sc.textFile(path).map(_.split(","))
                   .map{case Array(x1,x2) => (x1,x2)}
                   .collect.toList
}

loadHistogram: (path: String)List[(String, String)]


In [67]:
def hist2Plot(data: List[(String, String)], labelBins: String, labelValues: String) : List[Map[String, String]] = {
    data.map{case (bin, value) => Map(labelBins -> bin, labelValues -> value)}
  }

hist2Plot: (data: List[(String, String)], labelBins: String, labelValues: String)List[Map[String,String]]


In [124]:
//val MADplt=MAD.toList.map{case (x,y)=>Tuple2(x,y)}
val MADplt=MAD.sortBy(_._1).map(x => Tuple2(x._1.toString, x._2.toString)).toList
val BCNplt=BCN.sortBy(_._1).map(x => Tuple2(x._1.toString, x._2.toString)).toList
val AGPplt=AGP.sortBy(_._1).map(x => Tuple2(x._1.toString, x._2.toString)).toList

MADplt = List((1,24258), (2,22800), (3,24681), (4,25251), (5,26334), (6,22800), (7,22971), (8,21831), (9,21147), (10,22294), (11,20272), (12,14504))
BCNplt = List((1,29469), (2,28329), (3,30552), (4,31236), (5,28728), (6,26505), (7,29241), (8,27075), (9,23427), (10,20276), (11,19824), (12,15400))
AGPplt = List((1,9633), (2,8379), (3,10659), (4,8265), (5,10830), (6,7923), (7,8892), (8,7866), (9,8151), (10,6499), (11,6384), (12,3696))


List((1,9633), (2,8379), (3,10659), (4,8265), (5,10830), (6,7923), (7,8892), (8,7866), (9,8151), (10,6499), (11,6384), (12,3696))

In [125]:
val MADpltVegas = hist2Plot(MADplt, "Month", "# searches")
val BCNpltVegas = hist2Plot(BCNplt, "Month", "# searches")
val AGPpltVegas = hist2Plot(AGPplt, "Month", "# searches")

MADpltVegas = List(Map(Month -> 1, # searches -> 24258), Map(Month -> 2, # searches -> 22800), Map(Month -> 3, # searches -> 24681), Map(Month -> 4, # searches -> 25251), Map(Month -> 5, # searches -> 26334), Map(Month -> 6, # searches -> 22800), Map(Month -> 7, # searches -> 22971), Map(Month -> 8, # searches -> 21831), Map(Month -> 9, # searches -> 21147), Map(Month -> 10, # searches -> 22294), Map(Month -> 11, # searches -> 20272), Map(Month -> 12, # searches -> 14504))
BCNpltVegas = List(Map(Month -> 1, # searches -> 29469), Map(Month -> 2, # searches -> 28329), Map(Month -> 3, # searches -> 30552), Map(Month -> 4, # searches -> 31236), Map(Month -> 5, # searches -> 28728), Map(Month -> 6, # searches -> 26505), Map(Month -> 7, # se...


List(Map(Month -> 1, # searches -> 29469), Map(Month -> 2, # searches -> 28329), Map(Month -> 3, # searches -> 30552), Map(Month -> 4, # searches -> 31236), Map(Month -> 5, # searches -> 28728), Map(Month -> 6, # searches -> 26505), Map(Month -> 7, # searches -> 29241), Map(Month -> 8, # searches -> 27075), Map(Month -> 9, # searches -> 23427), Map(Month -> 10, # searches -> 20276), Map(Month -> 11, # searches -> 19824), Map(Month -> 12, # searches -> 15400))

In [127]:
//MAD
val searchesHist = Vegas("Number of Trips (Histogram)")
  .withData(MADpltVegas)
  .mark(Bar)
  .encodeY("# searches", Quantitative)
  .encodeX("Month", Quantitative)
searchesHist.show

<iframe id="frame-vegas-1284a43f-9823-47c6-8b9d-f8b34aebde73" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-1284a43f-9823-47c6-8b9d-f8b34aebde73'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "Month",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "# searches",
 "type" : "quantitative"
 }
 },
 "description" : "Number of Trips (Histogram)",
 "data" : {
 "values" : [
 {
 "Month" : "1",
 "# searches" : "24258"
 },
 {
 "Month" : "2",
 "# searches" : "22800"
 },
 {
 "Month" : "3",
 "# searches" : "24681"
 },
 {
 "Month" : "4",
 "# searches" : "25251"
 },
 {
 "Month" : "5",
 "# searches" : "26334"
 },
 {
 "Month" : "6",
 "# searches" : "22800"
 },
 {
 "Month" : "7",
 "# searches" : "22971"
 },
 {
 "Month" : "8",
 "# searches" : "21831"
 },
 {
 "Month" : "9",
 "# searches" : "21147"
 },
 {
 "Month" : "10",
 "# searches" : "22294"
 },
 {
 "Month" : "11",
 "# searches" : "20272"
 },
 {
 "Month" : "12",
 "# searches" : "14504"
 }
 ]
 }
}
 }
 vg.embed("#vegas-1284a43f-9823-47c6-8b9d-f8b34aebde73", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

searchesHist = ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Bar,Some(Encoding(None,None,Some(PositionChannelDef(None,None,None,Some(Month),Some(Quantitative),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(# searches),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None,None,None,None,None,None)),None,Some(Number of Trips (Histogram)),Some(Data(None,None,Some(List(Values(Map(Month -> 1, # searches -> 24258)), Values(Map(Month -> 2, # searches -> 22800)), Values(Map(Month -> 3, # searches -> 24681)), Values(Map(Month -> 4, # searches -> 25251)), Values(Map(Month -> 5, # searches -> 26334)), Values(Map(Month -> 6, # searches -> 22800)), Values(Map(Month -> 7, # searches -> 22971)), Values(Map(Month -> 8...


ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Bar,Some(Encoding(None,None,Some(PositionChannelDef(None,None,None,Some(Month),Some(Quantitative),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(# searches),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None,None,None,None,None,None)),None,Some(Number of Trips (Histogram)),Some(Data(None,None,Some(List(Values(Map(Month -> 1, # searches -> 24258)), Values(Map(Month -> 2, # searches -> 22800)), Values(Map(Month -> 3, # searches -> 24681)), Values(Map(Month -> 4, # searches -> 25251)), Values(Map(Month -> 5, # searches -> 26334)), Values(Map(Month -> 6, # searches -> 22800)), Values(Map(Month -> 7, # searches -> 22971)), Values(Map(Month -> 8, # searches -> 21831)), Values(Map(Month -> 9, # searches -> 21147)), Values(Map(Month -> 10, # searches -> 22294)), Values(Map(Month -> 11, # searches -> 20272)), Values(Map(Month -> 12, # searches -> 14504)))))),None,None))

In [132]:
//BCN
Vegas("Number of Trips (Histogram)")
  .withData(BCNpltVegas)
  .mark(Bar)
  .encodeY("# searches", Quantitative)
  .encodeX("Month", Quantitative)
   .show

<iframe id="frame-vegas-2af7cdae-e765-48b3-8051-9f4bd16664e9" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-2af7cdae-e765-48b3-8051-9f4bd16664e9'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "Month",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "# searches",
 "type" : "quantitative"
 }
 },
 "description" : "Number of Trips (Histogram)",
 "data" : {
 "values" : [
 {
 "Month" : "1",
 "# searches" : "29469"
 },
 {
 "Month" : "2",
 "# searches" : "28329"
 },
 {
 "Month" : "3",
 "# searches" : "30552"
 },
 {
 "Month" : "4",
 "# searches" : "31236"
 },
 {
 "Month" : "5",
 "# searches" : "28728"
 },
 {
 "Month" : "6",
 "# searches" : "26505"
 },
 {
 "Month" : "7",
 "# searches" : "29241"
 },
 {
 "Month" : "8",
 "# searches" : "27075"
 },
 {
 "Month" : "9",
 "# searches" : "23427"
 },
 {
 "Month" : "10",
 "# searches" : "20276"
 },
 {
 "Month" : "11",
 "# searches" : "19824"
 },
 {
 "Month" : "12",
 "# searches" : "15400"
 }
 ]
 }
}
 }
 vg.embed("#vegas-2af7cdae-e765-48b3-8051-9f4bd16664e9", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

In [144]:
//AGP
Vegas("Number of Trips (Histogram)")
  .withData(AGPpltVegas)
  .mark(Line)
  .encodeY("# searches", Quantitative)
  .encodeX("Month", Quantitative)
   .show

<iframe id="frame-vegas-d28283c0-be18-4a9c-89b2-325749a862ea" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-d28283c0-be18-4a9c-89b2-325749a862ea'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "Month",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "# searches",
 "type" : "quantitative"
 }
 },
 "description" : "Number of Trips (Histogram)",
 "data" : {
 "values" : [
 {
 "Month" : "1",
 "# searches" : "9633"
 },
 {
 "Month" : "2",
 "# searches" : "8379"
 },
 {
 "Month" : "3",
 "# searches" : "10659"
 },
 {
 "Month" : "4",
 "# searches" : "8265"
 },
 {
 "Month" : "5",
 "# searches" : "10830"
 },
 {
 "Month" : "6",
 "# searches" : "7923"
 },
 {
 "Month" : "7",
 "# searches" : "8892"
 },
 {
 "Month" : "8",
 "# searches" : "7866"
 },
 {
 "Month" : "9",
 "# searches" : "8151"
 },
 {
 "Month" : "10",
 "# searches" : "6499"
 },
 {
 "Month" : "11",
 "# searches" : "6384"
 },
 {
 "Month" : "12",
 "# searches" : "3696"
 }
 ]
 }
}
 }
 vg.embed("#vegas-d28283c0-be18-4a9c-89b2-325749a862ea", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">